In [201]:
import tensorflow as tf
import numpy as np
import scipy.special

In [202]:
N = 4
K = 2

In [203]:
def full_adder(a,b,c):
    s = (a ^ b) ^ c
    c = (a & b) | (c & (a ^ b))
    return s,c

def add_bool(a,b):
    k = len(a)
    s = np.zeros(k,dtype=bool)
    c = False
    for i in reversed(range(0,k)):
        s[i], c = full_adder(a[i],b[i],c)    
    
    return s

def inc_bin(a):
    k = len(a)
    increment = np.hstack((np.zeros(k-1,dtype=int), np.ones(1,dtype=int)))
    a = add_bool(a,increment)
    return a

def approx(C):
    W = inv_fc(1-(1-pi_fc(C))**2)  
    return W

def pi_fc(x):
    a = -0.4527
    b = 0.0218
    r = 0.86
    
    if x < 10:
        P = np.exp(a*(x**r)+b)
    else:
        P = np.sqrt((np.pi)/x)*np.exp(-x/4)*(1-10/(7*x))
        
    return P

def inv_fc(x):
    a = -0.4527
    b = 0.0218
    r = 0.86
    
    I = ((np.log(x)-b)/a)**(1/r)
    
    return I
    

def polarization(N, K, sigma_train):
    
    W = np.ones(N, dtype=float)
    W_temp = np.zeros(N, dtype=float)
    
    W[0] = (2/(sigma_train**2))
    
    for i in range(1, int(np.log2(N))+1):
        W_temp[:] = W[:]
        t = 2**(i-1)
        
        for j in range(1,(int(t))+1):
            C = W_temp[j-1]
            W[2*j-1-1] = approx(C)         #polarization by using density evolution
            W[2*j-1] = 2*C
                  
    qfunc = lambda x: 0.5-0.5*scipy.special.erf(x/np.sqrt(2))
    W = qfunc(np.sqrt(W/2))
    idx = sorted(range(W.size), key=lambda k: -W[k]) # descend order 
    A = idx[-K:]

    
    return A
    
def encoding(u, N):

    n = 1
    x = np.copy(u)
    step = np.log2(N)
    for s in range(0,step.astype(int)):
        i = 0
        while i < N:
            for j in range(0,n):
                idx = i+j
                x[idx] = x[idx] ^ x[idx+n]
            i=i+2*n
        n=2*n
        
    for s in range(0,N):
        if np.mod(x[s],2) == 0:
            x[s] = 0

    return x

In [204]:
snr_train = 2
sigma_train = np.sqrt(1/(2*10**(snr_train/10)))
print(sigma_train)

0.561674881261


In [205]:
b = np.zeros((2**K,K),dtype=int)
for i in range(1,2**K):
    b[i]= inc_bin(b[i-1])

In [206]:
A = polarization(N, K, sigma_train)
u = np.zeros((2**K, N), dtype=int)
u[:,A] = b

In [207]:
A

[2, 3]

In [208]:
x = np.zeros((2**K, N), dtype=int)

for i in range(0, 2**K):
    x[i,:] = encoding(u[i,:], N)
    
x = (-1)**x # BPSK modulation

In [209]:
b[:,:]

array([[0, 0],
       [0, 1],
       [1, 0],
       [1, 1]])

In [210]:
u[:,:]

array([[0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 1, 1]])

In [211]:
x[:,:]

array([[ 1,  1,  1,  1],
       [-1, -1, -1, -1],
       [-1,  1, -1,  1],
       [ 1, -1,  1, -1]])

In [212]:
Layer1 = N
Layer2 = N
Layer3 = N
Layer4 = N


X = tf.placeholder(tf.float32,[None, N])
Y = tf.placeholder(tf.float32,[None, N])

W1 = tf.Variable(tf.random_uniform([N, Layer1], -1.0, 1.0))
W2 = tf.Variable(tf.random_uniform([Layer1, Layer2], -1.0, 1.0))
W3 = tf.Variable(tf.random_uniform([Layer2, Layer3], -1.0, 1.0))
W4 = tf.Variable(tf.random_uniform([Layer3, N], -1.0, 1.0))

b1 = tf.Variable(tf.zeros([Layer1]), name = "Bias1")
b2 = tf.Variable(tf.zeros([Layer2]), name = "Bias2")
b3 = tf.Variable(tf.zeros([Layer3]), name = "Bias3")
b4 = tf.Variable(tf.zeros([Layer4]), name = "Bias4")

L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
hypo = tf.nn.sigmoid(tf.matmul(L3, W4) + b4)

#cost = -tf.reduce_mean(tf.maximum(Y,0)*tf.log(hypo)+tf.maximum(-Y,0)*tf.log(-hypo)+((1-tf.abs(Y))*tf.log(1-hypo)))
#cost = -tf.reduce_mean(tf.maximum(Y,0)*tf.log(hypo)+tf.maximum(-Y,0)*tf.log(-hypo)+((1-tf.abs(Y))*tf.log(1-hypo)))
#cost = -tf.reduce_mean(Y*tf.log(hypo)+(1-Y)*tf.log(1-hypo))
cost = tf.reduce_mean(tf.square(Y-hypo))

a = 0.01
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

init = tf.global_variables_initializer()

In [213]:
sess = tf.Session()
sess.run(init)

for step in range(2**16):
    sess.run(train, feed_dict={X:x+np.random.normal(0,sigma,(2**K, N)), Y:u})
    
    if step % 1000 == 0:
        print(step)
        print(sess.run(cost, feed_dict={X:x+np.random.normal(0,sigma,(2**K, N)), Y:u})) 
        #print(sess.run(W1, feed_dict={X:x, Y:u}))
        
#correct_prediction = tf.equal(tf.round(2*hypo-1), Y)
correct_prediction = tf.equal(tf.round(hypo), Y)

accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))
print("Accuracy:")
print(sess.run(accuracy, feed_dict={X:x+np.random.normal(0,sigma,(2**K, N)), Y:u}))


0
0.198553
1000
0.149134
2000
0.129354
3000
0.120017
4000
0.126486
5000
0.0872497
6000
0.084933
7000
0.0610006
8000
0.0603338
9000
0.0523601
10000
0.0592973
11000
0.0526446
12000
0.0478151
13000
0.0372588
14000
0.0392708
15000
0.062213
16000
0.0389597
17000
0.0526303
18000
0.0392722
19000
0.0342201
20000
0.0335977
21000
0.0325852
22000
0.0323806
23000
0.0814263
24000
0.0253377
25000
0.0312647
26000
0.0376932
27000
0.0173976
28000
0.0312127
29000
0.0143645
30000
0.0140978
31000
0.0141563
32000
0.0120752
33000
0.0119091
34000
0.11043
35000
0.00984312
36000
0.0187313
37000
0.0108363
38000
0.0517202
39000
0.117899
40000
0.00634809
41000
0.0431389
42000
0.00886906
43000
0.0106179
44000
0.00708729
45000
0.00573385
46000
0.0212009
47000
0.0179291
48000
0.00449727
49000
0.00422526
50000
0.0040425
51000
0.00397039
52000
0.00625772
53000
0.00500973
54000
0.00367481
55000
0.0557706
56000
0.00340157
57000
0.00656313
58000
0.0053603
59000
0.00339337
60000
0.00319816
61000
0.00369629
62000
0.0029662